In [2]:
pip install sacrebleu rouge-score

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.8/51.8 kB 3.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 104.0/104.0 kB 7.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.0/5.0 MB 65.1 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 74.4 MB/s eta 0:00:00
  Created wheel for rouge-score: filename=rouge_score-0.1.2-py3-none-any.whl size=24934 sha256=f178902d6a5bfbea5b36cb332b483c5982a11ca9d57b6b99fb5f6c1b7747553c
  Stored in directory: /home/jupyter/.cache/pip/wheels/5f/dd/89/461065a73be61a532ff8599a28e9beef17985c9e9c31e541b4
Successfully built rouge-score
Note: you may need to restart the kernel to use updated packages.


In [11]:
pip install -U transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.9/9.9 MB 58.0 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 104.0 MB/s eta 0:00:00
  Attempting uninstall: tokenizers
    Found existing installation: tokenizers 0.19.1
    Uninstalling tokenizers-0.19.1:
      Successfully uninstalled tokenizers-0.19.1
  Attempting uninstall: transformers
    Found existing installation: transformers 4.44.2
    Uninstalling transformers-4.44.2:
      Successfully uninstalled transformers-4.44.2
Note: you may need to restart the kernel to use updated packages.


In [ ]:
import os
import json
import torch
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer
from sacrebleu import corpus_bleu
from rouge_score import rouge_scorer
from google.cloud import storage

# Check if CUDA is available
device = 'cuda' if torch.cuda.is_available() else 'cpu'

# Path to the checkpoint folder in GCS
checkpoint_path = 'gs://translation-datasets/nllb-finetuned/nllb-finetuned/checkpoint-72000'
local_checkpoint_path = '/tmp/checkpoint-72000'

# Function to download model files from GCS to local storage
def download_model_from_gcs(gcs_path, local_path):
    client = storage.Client()
    bucket_name, folder_path = gcs_path.replace('gs://', '').split('/', 1)
    bucket = client.bucket(bucket_name)
    blobs = bucket.list_blobs(prefix=folder_path)

    os.makedirs(local_path, exist_ok=True)

    for blob in blobs:
        local_file_path = os.path.join(local_path, blob.name.split('/')[-1])
        blob.download_to_filename(local_file_path)
        print(f"Downloaded {blob.name} to {local_file_path}")

# Download the model from GCS
download_model_from_gcs(checkpoint_path, local_checkpoint_path)

# Load the fine-tuned model and tokenizer from the local path
model = AutoModelForSeq2SeqLM.from_pretrained(local_checkpoint_path)
tokenizer = AutoTokenizer.from_pretrained(local_checkpoint_path)

# Move the model to the appropriate device, The device is determined based on whether a GPU is available 'cuda' or not 'cpu'
model = model.to(device)

# Function to load the JSONL test data from GCS and limit to 10 examples
def load_jsonl(file_path, max_examples=30):
    test_data = []
    client = storage.Client()
    bucket_name, blob_name = file_path.replace('gs://', '').split('/', 1)
    bucket = client.bucket(bucket_name)
    blob = bucket.blob(blob_name)
    content = blob.download_as_text(encoding='utf-8')

    # Process the content and limit to max_examples
    for i, line in enumerate(content.splitlines()):
        if i >= max_examples:
            break
        test_data.append(json.loads(line))
    return test_data

#def load_jsonl(file_path):
#    test_data = []
#    client = storage.Client()
#    bucket_name, blob_name = file_path.replace('gs://', '').split('/', 1)
#    bucket = client.bucket(bucket_name)
#    blob = bucket.blob(blob_name)
#    content = blob.download_as_text(encoding='utf-8')

    # Process the content and load all examples
#    for line in content.splitlines():
#        test_data.append(json.loads(line))
    
#    return test_data

# Function to perform translation
def translate_text(model, tokenizer, text, source_lang_code, target_lang_code):
    tokenizer.src_lang = source_lang_code
    inputs = tokenizer(text, return_tensors="pt", padding=True, truncation=True).to(device)

    # Generate translation
    forced_bos_token_id = tokenizer.convert_tokens_to_ids(target_lang_code)
    generated_tokens = model.generate(**inputs, forced_bos_token_id=forced_bos_token_id)
    translated_text = tokenizer.batch_decode(generated_tokens, skip_special_tokens=True)[0]

    return translated_text

# Load the test dataset
test_data_path = 'gs://translation-datasets/test.jsonl'
test_data = load_jsonl(test_data_path,max_examples=30)

# Perform translation and store results
reference_texts = []
generated_texts = []

for sample in test_data:
    source_text = sample['source_text']
    target_text = sample['target_text']
    source_lang_code = sample['source_lang']
    target_lang_code = sample['target_lang']

    # Perform translation
    translated_text = translate_text(model, tokenizer, source_text, source_lang_code, target_lang_code)

    # Store the reference and generated texts for evaluation
    reference_texts.append(target_text)
    generated_texts.append(translated_text)

# Function to evaluate BLEU score
def evaluate_bleu(reference_texts, generated_texts):
    bleu = corpus_bleu(generated_texts, [reference_texts])
    return bleu.score

# Function to evaluate ROUGE scores
def evaluate_rouge(reference_texts, generated_texts):
    scorer = rouge_scorer.RougeScorer(['rouge1', 'rouge2', 'rougeL'], use_stemmer=True)
    rouge_scores = []

    for ref, gen in zip(reference_texts, generated_texts):
        score = scorer.score(ref, gen)
        rouge_scores.append(score)

    # Return the average ROUGE scores
    avg_rouge1 = sum([score['rouge1'].fmeasure for score in rouge_scores]) / len(rouge_scores)
    avg_rouge2 = sum([score['rouge2'].fmeasure for score in rouge_scores]) / len(rouge_scores)
    avg_rougeL = sum([score['rougeL'].fmeasure for score in rouge_scores]) / len(rouge_scores)

    return {
        "ROUGE-1": avg_rouge1,
        "ROUGE-2": avg_rouge2,
        "ROUGE-L": avg_rougeL
    }

# Evaluate BLEU score
bleu_score = evaluate_bleu(reference_texts, generated_texts)
print(f"BLEU score: {bleu_score}")

# Evaluate ROUGE scores
rouge_scores = evaluate_rouge(reference_texts, generated_texts)
for metric, score in rouge_scores.items():
    print(f"{metric}: {score}")

In [5]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
import torch

local_checkpoint_path = '/tmp/checkpoint-72000'

model = AutoModelForSeq2SeqLM.from_pretrained(local_checkpoint_path)
tokenizer = AutoTokenizer.from_pretrained(local_checkpoint_path)

device = 'cuda' if torch.cuda.is_available() else 'cpu'
model = model.to(device)

source_lang = "eng_Latn"
target_lang = "fra_Latn"

def translate_batch_nllb(texts, source_lang, target_lang):

    tokenizer.src_lang = source_lang

    inputs = tokenizer(texts, return_tensors="pt", padding=True, truncation=True).to(device)

    forced_bos_token_id = tokenizer.convert_tokens_to_ids(target_lang)

    generated_tokens = model.generate(**inputs, forced_bos_token_id=forced_bos_token_id)

    translations = tokenizer.batch_decode(generated_tokens, skip_special_tokens=True)
    return translations
 
texts = [
    "Hello, how are you?",
    "This is a test of the NLLB model.",
    "Let's see how well it performs."
]

translated_texts = translate_batch_nllb(texts, source_lang, target_lang)

for i, translation in enumerate(translated_texts):
    print(f"Original: {texts[i]}")
    print(f"Translated: {translation}\n")

Original: Hello, how are you?
Translated: Bonjour, comment allez-vous?

Original: This is a test of the NLLB model.
Translated: Ceci est un test du modèle NLLB.

Original: Let's see how well it performs.
Translated: Voyons si ça marche.

